In [7]:
import pandas as pd
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

train = pd.read_json('/Users/marcusdemaster/Documents/MIDS/w207/FinalProject/train.json', orient='columns')
test = pd.read_json('/Users/marcusdemaster/Documents/MIDS/w207/FinalProject/test.json', orient='columns')

In [10]:
np_test = np.array(test)

print np_test

[[u'N/A' u't3_i8iy4'
  u"Hey all! It's about 95 degrees here and our kitchen is pretty much empty save for some bread and cereal.  My girlfriend/fiance is 8 1/2 months pregnant and we could use a good meal.  We promise to pay it forward when we get money! Thanks so much in advance!"
  ..., u'j_like' 1308963419 1308959819]
 [u'N/A' u't3_1mfqi0'
  u"I didn't know a place like this exists! \n\nI was out sick for 3 days the week before Labour Day. After my manager came back, I was let go at the end of my shift. I was supposed to be paid my last pay(big whopping 7.5 hours of pay) on Friday that just passed, and of course, they didn't pay me and said that'd be another week before I'll get my final check from them. I start at a new job tomorrow but won't be paid until October 4th and that money all goes right into rent. \n\nI just would love to eat pizza and watch Breaking Bad tonight as if everything was normal. "
  ..., u'0110110101101100' 1379263523 1379259923]
 [u'N/A' u't3_lclka'
  u'Hi 